# Transformations du bronze en Silver format sql

Ce notebook effectue des transformations sur les données du lakehouse Bronze et enregistre les données de transformation dans le lakehouse Silver.

In [ ]:
%%sql
-- Creer une vue temporaire de la table wind_power_production
CREATE OR REPLACE TEMPORARY VIEW bronze_wind_power_production AS
SELECT *
FROM LH_Bronze.wind_power_production

In [ ]:
%%sql
-- Nettoyer et enrichir les données
CREATE OR REPLACE TEMPORARY VIEW transformed_wind_power_production AS
SELECT
    production_id,date,turbine_name,capacity,location_name,latitude,longitude,
    region,status,responsible_department,
    wind_direction,
    ROUND(wind_speed, 2) AS wind_speed,
    ROUND(energy_produced, 2) AS energy_produced,
    DAY(date) AS day,
    MONTH(date) AS month,
    QUARTER(date) AS quarter,
    YEAR(date) as year,
    REGEXP_REPLACE(time, '-', ':') AS time,
    CAST(SUBSTRING(time, 1, 2) AS INT) AS hour_of_day,
    CAST(SUBSTRING(time, 4, 2) AS INT) AS minute_of_hour,
    CAST(SUBSTRING(time, 7, 2) AS INT) AS second_of_minute,
    CASE
        WHEN CAST(SUBSTRING(time, 1, 2) AS INT) BETWEEN 5 AND 11 THEN 'Morning'
        WHEN CAST(SUBSTRING(time, 1, 2) AS INT) BETWEEN 12 AND 16 THEN 'Afternoon'
        WHEN CAST(SUBSTRING(time, 1, 2) AS INT) BETWEEN 17 AND 20 THEN 'Evening'
        ELSE 'Night'
    END AS time_period
FROM bronze_wind_power_production;

In [ ]:
%%sql
-- Supprimez la table wind_power_production dans Lakehouse Silver  si elle existe
DROP TABLE IF EXISTS LH_Silver.wind_power_production;

In [ ]:
%%sql
-- Créer la nouvelle table wind_power_production dans Lakehouse Silver
CREATE TABLE LH_Silver.wind_power_production--creation d'une nouvelle table
USING delta --au format deltat
AS
SELECT * FROM transformed_wind_power_production --dont le contenu de la table